In [1]:
from sklearn import svm, datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
import pandas as pd

In [2]:
parameters = {'max_depth':[1,5,10, None], 'min_samples_leaf':[1, 2,3,4]}
d_clf = DecisionTreeClassifier()
clf = GridSearchCV(d_clf, parameters,cv=6)
clf.fit(iris.data, iris.target)

/Users/paulbruffett/anaconda3/envs/universe/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=6, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [1, 5, 10, None],
                         'min_samples_leaf': [1, 2, 3, 4]},
             pre_dis

In [3]:
clf.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [4]:
pd.DataFrame(clf.cv_results_)[['mean_test_score','params']].sort_values('mean_test_score',ascending=False)

,mean_test_score,params
4,0.966667,"{'max_depth': 5, 'min_samples_leaf': 1}"
5,0.960000,"{'max_depth': 5, 'min_samples_leaf': 2}"
6,0.960000,"{'max_depth': 5, 'min_samples_leaf': 3}"
7,0.960000,"{'max_depth': 5, 'min_samples_leaf': 4}"
8,0.960000,"{'max_depth': 10, 'min_samples_leaf': 1}"
9,0.960000,"{'max_depth': 10, 'min_samples_leaf': 2}"
10,0.960000,"{'max_depth': 10, 'min_samples_leaf': 3}"
11,0.960000,"{'max_depth': 10, 'min_samples_leaf': 4}"
12,0.960000,"{'max_depth': None, 'min_samples_leaf': 1}"
13,0.960000,"{'max_depth': None, 'min_samples_leaf': 2}"


In [5]:
import os
import pandas as pd
import time
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import (
    RandomParameterSampling, choice, PrimaryMetricGoal,
    HyperDriveConfig, MedianStoppingPolicy)
#from azureml.widgets import RunDetails
import azureml.core
#from get_auth import get_auth
print('azureml.core.VERSION={}'.format(azureml.core.VERSION))

azureml.core.VERSION=1.0.83


In [6]:
directory_id = 'e2854f21-58a1-47c9-ab4a-26baaf2e34b8'
app_id = '60e2cfe4-b3cf-4512-a580-494ba4814dfb'
app_key = '5YB/z_1pH]MAR2E_xBTTSkRMokV49.?]'

In [7]:
auth = azureml.core.authentication.ServicePrincipalAuthentication(directory_id,app_id,app_key)

In [8]:
ws = Workspace('3bdbda93-8c3a-472b-bdde-25e3028fc307','mlworkspace','pbml',auth=auth)
ws_details = ws.get_details()
print('Name:\t\t{}\nLocation:\t{}'
      .format(ws_details['name'],
              ws_details['location']))

Name:		pbml
Location:	eastus2


In [9]:
ws.get_details()

{'id': '/subscriptions/3bdbda93-8c3a-472b-bdde-25e3028fc307/resourceGroups/mlworkspace/providers/Microsoft.MachineLearningServices/workspaces/pbml',
 'name': 'pbml',
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': 'b535df4b-9bd2-4e9c-b96c-ff4910d6b94b',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-09-27T16:10:32.3772866+00:00',
 'keyVault': '/subscriptions/3bdbda93-8c3a-472b-bdde-25e3028fc307/resourcegroups/mlworkspace/providers/microsoft.keyvault/vaults/pbml9532469074',
 'applicationInsights': '/subscriptions/3bdbda93-8c3a-472b-bdde-25e3028fc307/resourcegroups/mlworkspace/providers/microsoft.insights/components/pbml8394139910',
 'identityPrincipalId': '61a3db8b-46eb-4e91-86e2-68512d76219e',
 'identityTenantId': 'e2854f21-58a1-47c9-ab4a-26baaf2e34b8',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/3bdbda93-8c3a-472b-bdde-25e3028fc307/resourcegroups/mlworkspace/p

In [63]:
cluster_name = 'hypetuning'
provisioning_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D4_v2',
        # vm_priority = 'lowpriority', # optional
        max_nodes=16)

In [64]:
ds = ws.get_default_datastore()

In [72]:
ds.

$AZUREML_DATAREFERENCE_workspaceblobstore

In [61]:
hyperparameter_sampling = RandomParameterSampling({
    'ngrams': choice(range(1, 5)),
    'match': choice(range(2, 41)),
    'min_child_samples': choice(range(1, 31)),
    'unweighted': choice('Yes', 'No')
})


In [62]:
from azureml.train.hyperdrive import HyperDriveConfig
hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                          hyperparameter_sampling=param_sampling, 
                          policy=early_termination_policy,
                          primary_metric_name="accuracy", 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs=100,
                          max_concurrent_runs=4)

TypeError: __init__() missing 1 required positional argument: 'source_directory'